# Task 1: Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

In [1]:
!pip3 install sklearn-crfsuite


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import the necessary libraries

In [2]:
import nltk
from nltk import word_tokenize, pos_tag
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
import sklearn_crfsuite

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bishn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bishn\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [3]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [4]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [5]:
def tuples_maker(inp, out):
    j=0
    sample_data = []
    for sent in inp:
        k=0
        sentence_tag = []
        for word in sent:
            sentence_tag.append((word,out[j][k]))
            k+=1
        sample_data.append(sentence_tag)
        j+=1
    return sample_data

In [6]:
whole_data = tuples_maker(input_sigs, output_labels)

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [7]:
def triples_maker(whole_data):    
    sample_data = []
    for sent in whole_data:
        sentence = []
        pos_tags = pos_tag([word[0] for word in sent])
        i = 0
        for word in sent:
            sentence.append((word[0], pos_tags[i][1], word[1]))
            i+=1
        sample_data.append(sentence)
    return sample_data

In [8]:
pos_tag(input_sigs[0])

[('for', 'IN'), ('5', 'CD'), ('to', 'TO'), ('6', 'CD'), ('days', 'NNS')]

In [9]:
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [10]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    # Common features for all words
    features = {
        'bias': 0,
        'word.lower': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper': word.isupper(),
        'word.istitle': word.istitle(),
        'word.isdigit': word.isdigit(),
        'postag': postag
    }

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features['-1:word.lower'] = word1.lower()
        features['-1:word.istitle'] = word1.istitle()
        features['-1:word.isupper'] = word1.isupper()
        features['-1:word.isdigit'] = word1.isdigit()
        features['-1:postag'] = postag1
    else:
        # Indicate that it is the 'beginning of a document'
        features['BOS'] = True
    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features['+1:word.lower'] = word1.lower()
        features['+1:word.istitle'] = word1.istitle()
        features['+1:word.isupper'] = word1.isupper()
        features['+1:word.isdigit'] = word1.isdigit()
        features['+1:postag'] = postag1
    else:
        # Indicate that it is the 'end of a document'
        features['EOS'] = True

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [11]:
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

In [12]:
def sent2features(sent):
    return [token_to_features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, postag, label in sent]
def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [13]:
X = [sent2features(sent) for sent in sample_data]
y = [sent2labels(sent) for sent in sample_data]


In [14]:
# vectorizer = DictVectorizer(sparse=False)  # Set sparse=True for a sparse matrix

# X = vectorizer.fit_transform(X)
X


[[{'bias': 0,
   'word.lower': 'for',
   'word[-3:]': 'for',
   'word[-2:]': 'or',
   'word.isupper': False,
   'word.istitle': False,
   'word.isdigit': False,
   'postag': 'IN',
   'BOS': True,
   '+1:word.lower': '5',
   '+1:word.istitle': False,
   '+1:word.isupper': False,
   '+1:word.isdigit': True,
   '+1:postag': 'CD'},
  {'bias': 0,
   'word.lower': '5',
   'word[-3:]': '5',
   'word[-2:]': '5',
   'word.isupper': False,
   'word.istitle': False,
   'word.isdigit': True,
   'postag': 'CD',
   '-1:word.lower': 'for',
   '-1:word.istitle': False,
   '-1:word.isupper': False,
   '-1:word.isdigit': False,
   '-1:postag': 'IN',
   '+1:word.lower': 'to',
   '+1:word.istitle': False,
   '+1:word.isupper': False,
   '+1:word.isdigit': False,
   '+1:postag': 'TO'},
  {'bias': 0,
   'word.lower': 'to',
   'word[-3:]': 'to',
   'word[-2:]': 'to',
   'word.isupper': False,
   'word.istitle': False,
   'word.isdigit': False,
   'postag': 'TO',
   '-1:word.lower': '5',
   '-1:word.istitle':

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

### Training the CRF model with the features extracted using the feature extractor method

In [25]:

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=1000,
    all_possible_transitions=True,  # Allow all possible label transitions
    model_filename= 'crf_model.pkl',
)
# Submit training data to the trainer


# Set the parameters of the model


# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
predictions = crf.predict(X_test)


In [26]:
y_test[0]

['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit']

In [27]:
crf.predict_single(X_test[0])

['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']

### Predicting the test data with the built model

### Putting all the prediction logic inside a predict method

In [28]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    words = sig.split()
    pos_tags = pos_tag(words)
    processed_data = sent2features(pos_tags)
    predictions = crf.predict_single(processed_data)
    print(sig)
    print(predictions)
    return predictions

### Sample predictions

In [29]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

take 2 tabs every 6 hours x 10 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [30]:
predictions = predict("2 capsu for 10 day at bed")

2 capsu for 10 day at bed
['Qty', 'Form', 'FOR', 'Duration', 'PeriodUnit', 'AT', 'WHEN']


In [31]:
predictions = predict("2 capsu for 10 days at bed")

2 capsu for 10 days at bed
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [32]:
predictions = predict("5 days 2 tabs at bed")

5 days 2 tabs at bed
['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']


In [33]:
predictions = predict("3 tabs qid x 10 weeks")

3 tabs qid x 10 weeks
['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']


In [34]:
predictions = predict("x 30 days")

x 30 days
['FOR', 'Duration', 'DurationUnit']


In [35]:
predictions = predict("x 20 months")

x 20 months
['FOR', 'Duration', 'DurationUnit']


In [36]:
predictions = predict("take 2 tabs po tid for 10 days")

take 2 tabs po tid for 10 days
['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']


In [37]:
predictions = predict("take 2 capsules po every 6 hours")

take 2 capsules po every 6 hours
['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']


In [38]:
predictions = predict("inject 2 units pu tid")

inject 2 units pu tid
['Method', 'Qty', 'Form', 'PO', 'TID']


In [39]:
predictions = predict("swallow 3 caps tid by mouth")

swallow 3 caps tid by mouth
['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']


In [40]:
predictions = predict("inject 3 units orally")

inject 3 units orally
['Method', 'Qty', 'Form', 'PO']


In [41]:
predictions = predict("orally take 3 tabs tid")

orally take 3 tabs tid
['PO', 'Method', 'Qty', 'Form', 'TID']


In [42]:
predictions = predict("by mouth take three caps")

by mouth take three caps
['BY', 'PO', 'Method', 'Qty', 'Form']


In [43]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

take 3 tabs orally three times a day for 10 days at bedtime
['Method', 'Qty', 'Form', 'PO', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [44]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']


In [45]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
['Method', 'Qty', 'Form', 'Frequency', 'DAILY', 'AT', 'WHEN']


In [46]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']


In [47]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']


In [48]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']


In [49]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
['Method', 'Method', 'Qty', 'Form']


In [50]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']


In [51]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [52]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']


In [53]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
['Method', 'Qty', 'Form']


In [54]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
['Method', 'Qty', 'TO', 'Qty', 'Form']


In [55]:
prediction = predict("take two to four tabs")

take two to four tabs
['Method', 'Qty', 'TO', 'Qty', 'Form']


In [56]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']


In [57]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [58]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [59]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']


In [60]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [61]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [62]:
prediction = predict("every 60 mins")

every 60 mins
['EVERY', 'Period', 'PeriodUnit']


In [63]:
prediction = predict("every 10 mins")

every 10 mins
['EVERY', 'Period', 'PeriodUnit']


In [64]:
prediction = predict("every two to four months")

every two to four months
['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [65]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']


In [66]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
['EVERY', 'Period', 'TO', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form']


In [67]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
['Frequency', 'TIMES', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']


In [68]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
['Method', 'Qty', 'Form', 'Frequency', 'TIMES', 'EVERY', 'Period', 'PeriodUnit']


In [69]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']


In [70]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [71]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']


In [72]:
prediction = predict("10 tabs")

10 tabs
['Qty', 'Form']
